In [15]:
import cv2
import numpy as np

DB_path = './data/VOC2007_trainval'

id = 'JPEGImages/000017.jpg'
img = cv2.imread(f'{DB_path}/{id}')

bnd = '185 62 279 199,90 78 403 336'
cls = '14,12'
bnd = list(map(str, bnd.split(',')))
cls = list(map(int, cls.split(',')))

for bbox in bnd:
    box = list(map(int, bbox.split(' ')))
    img = cv2.rectangle(img, (box[0], box[1]), (box[2], box[3]), (255, 0, 0), 2)

cv2.imshow('t', img)
cv2.waitKey()
cv2.destroyAllWindows()



In [119]:
import torch
import os
import cv2
import pandas as pd

DB_path = './data/VOC2007_trainval'

label_csv = pd.read_csv('2007_train.csv')
for idx in range(5):
    img_path = os.path.join(f'{DB_path}', label_csv.iloc[idx, 0])

    bboxs = label_csv.iloc[idx, 1]
    bboxs = list(map(str, bboxs.split(',')))
    bboxs = np.array(bboxs)
    bbox = []
    for bx in bboxs:
        bbox.append(list(map(int, bx.split(' '))))
    bbox = np.array(bbox)

    clss = label_csv.iloc[idx, 2]
    clss = list(map(int, clss.split(',')))
    clss = np.array(clss)
    sample = {'image': img_path, 'Bboxs': bbox, 'Classes': clss}

    print(sample)
#%

{'image': './data/VOC2007_trainval\\JPEGImages/000012.jpg', 'Bboxs': array([[156,  97, 351, 270]]), 'Classes': array([6])}
{'image': './data/VOC2007_trainval\\JPEGImages/000017.jpg', 'Bboxs': array([[185,  62, 279, 199],
       [ 90,  78, 403, 336]]), 'Classes': array([14, 12])}
{'image': './data/VOC2007_trainval\\JPEGImages/000023.jpg', 'Bboxs': array([[  9, 230, 245, 500],
       [230, 220, 334, 500],
       [  2, 178,  90, 500],
       [  2,   1, 117, 369],
       [  3,   2, 243, 462],
       [225,   1, 334, 486]]), 'Classes': array([ 1,  1,  1, 14, 14, 14])}
{'image': './data/VOC2007_trainval\\JPEGImages/000026.jpg', 'Bboxs': array([[ 90, 125, 337, 212]]), 'Classes': array([6])}
{'image': './data/VOC2007_trainval\\JPEGImages/000032.jpg', 'Bboxs': array([[104,  78, 375, 183],
       [133,  88, 197, 123],
       [195, 180, 213, 229],
       [ 26, 189,  44, 238]]), 'Classes': array([ 0,  0, 14, 14])}


In [117]:
import torch
import os
import pandas as pd
from torch.utils.data import Dataset, DataLoader


DB_path = './data/VOC2007_trainval'
csv_file = '2007_train.csv'

class CustomDataset(Dataset):
    def __init__(self):
        self.label_csv = pd.read_csv('2007_train.csv')

    def __len__(self):
        return len(self.label_csv)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_path = os.path.join(f'{DB_path}', self.label_csv.iloc[idx, 0])

        bboxs = label_csv.iloc[idx, 1]
        bboxs = list(map(str, bboxs.split(',')))
        bboxs = np.array(bboxs)
        bbox = []
        for bx in bboxs:
            bbox.append(list(map(int, bx.split(' '))))
        bbox = np.array(bbox)

        clss = label_csv.iloc[idx, 2]
        clss = list(map(int, clss.split(',')))
        clss = np.array(clss)

        sample = {'image': img_path, 'Bboxs': bbox, 'Classes': clss}

        return sample


CustomDataset

__main__.CustomDataset